In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from numpy.linalg import inv
import matplotlib.pyplot as plt
from math import floor, log
import os

不得不说 sb 是真的多。

我笨，但不是 sb。

然后归纳一下机器学习步骤。

1. 建立一个模型，也就是一个函数式子
2. 找一个 loss function，衡量预测值与真实值之间的误差
3. 通过一个方法，使得预测值最接近真实值，由此得到的参数，来补全模型。

以上是大体情况。

具体到代码实践，应该还有个第 ```0``` 步。

0. 数据预处理，让数据更方便用矩阵向量等来表示。目前有用到 one-hot 编码。

# 源数据处理

由于源 ```csv``` 存在一些争议数据和多余数据，就直接处理，就可以不用管这一步的操作。

In [2]:
# 测试集
train_data = pd.read_csv('data/train.csv')
train_data = train_data.rename(columns = { 'native_country' : 'native_country&area' })
need_rm    = [i for i, ca in enumerate(train_data['native_country&area']) if ca == ' Holand-Netherlands' ]
train_data = train_data.drop(need_rm)


train_data

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country&area,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# 数据预处理
数据需要做一些处理，才合适放入模型投喂。

先搞 ```x, y``` 数据

In [3]:
def x_data_treatment(train_data, is_train):
    # 本函数由于用于训练集与测试集，所以需要区分
    # 测试集需要删除 income 列
    # sex 删除的原因是，后头会使用 one-hot 编码，sex 只有两个值，可以直接用 0 1 标识男女。
    if is_train:
        x_data = train_data.drop(['sex', 'income'], axis = 1)
    else:
        x_data = train_data.drop(['sex'], axis = 1)
        
    # 将纯数值与非纯数值的 col 分开。
    value_cols  = [ col for col in x_data.columns if x_data[col].dtypes == 'int' ]
    obj_cols    = [ col for col in x_data.columns if col not in value_cols ]
    value_frame = x_data[value_cols]
    obj_frame   = x_data[obj_cols]

    # sex 整成 01 男 1 女 0
    sex_col_frame = (train_data.sex == ' Male').astype(np.int)
    
    # sex 加入数值大家庭
    value_frame.insert(0, 'sex', sex_col_frame)
    obj_frame = pd.get_dummies(obj_frame)
    x_train = pd.concat([value_frame, obj_frame], axis = 1)
    x_train = x_train.astype(np.int)
    
    #x_train = pd.get_dummies(x_train) # one-hot 编码 但只针对非数值字段啊我操
    x_train = (x_train - x_train.mean()) / x_train.std() # 归一化操作
    return x_train


x_train = x_data_treatment(train_data, True)
x_train

,sex,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,...,native_country&area_ Portugal,native_country&area_ Puerto-Rico,native_country&area_ Scotland,native_country&area_ South,native_country&area_ Taiwan,native_country&area_ Thailand,native_country&area_ Trinadad&Tobago,native_country&area_ United-States,native_country&area_ Vietnam,native_country&area_ Yugoslavia
0,0.703028,0.030655,-1.063664,1.134703,0.148444,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
1,0.703028,0.837071,-1.008760,1.134703,-0.145921,-0.216583,-2.222086,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
2,0.703028,-0.042656,0.245033,-0.420048,-0.145921,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
3,0.703028,1.057003,0.425757,-1.197423,-0.145921,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
4,-1.422375,-0.775762,1.408136,1.134703,-0.145921,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,-2.933335,-0.045408,-0.022173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-1.422375,-0.849072,0.639698,0.746015,-0.145921,-0.216583,-0.197404,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
32557,0.703028,0.103966,-0.335482,-0.420048,-0.145921,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
32558,-1.422375,1.423556,-0.358826,-0.420048,-0.145921,-0.216583,-0.035429,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173
32559,0.703028,-1.215625,0.110913,-0.420048,-0.145921,-0.216583,-1.655175,-0.24445,-0.174295,-0.262098,...,-0.033729,-0.059274,-0.019201,-0.049628,-0.039607,-0.023518,-0.024163,0.340898,-0.045408,-0.022173


训练集的输入 feature (也就是 X) 已经处理，现在处理输出值

In [4]:
def y_data_treatment(train_data):
    # <= 50k 为 0
    # > 50k 为 1
    y_train = (train_data['income'] == ' >50K').astype('int')
    #print(y_train)
    return y_train

y_train = y_data_treatment(train_data)
y_train

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: income, Length: 32560, dtype: int64

## 分割训练集

In [5]:
def split_valid(x_train, y_train, valid_percent):
    
    X_train           = x_train.values
    Y_train           = y_train.values
    row_num           = X_train.shape[0]
    #print(X_train)
    valid_num         = int(row_num * valid_percent)
    need_random_range = np.arange(row_num)
    np.random.shuffle(need_random_range)
    X_train_valid_set, X_train_set = X_train[need_random_range[:valid_num]], X_train[need_random_range[valid_num:]]
    Y_train_valid_set, Y_train_set = Y_train[need_random_range[:valid_num]], Y_train[need_random_range[valid_num:]]
    #print(X_train[need_random_range[:valid_num]])
    return X_train_valid_set, X_train_set, Y_train_valid_set, Y_train_set
    
X_train_valid_set, X_train_set, Y_train_valid_set, Y_train_set = split_valid(x_train, y_train, 0.1)
Y_train_set

array([0, 0, 0, ..., 0, 0, 0])

# 直接训练

## sigmoid

In [6]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-8, (1-(1e-8)))

In [18]:
def train(X_train, Y_train):
    row_num     = X_train.shape[0]
    feature_num = X_train.shape[1]
    w           = np.zeros(feature_num)
    b           = 0
    l_rate      = 0.00001 # 需要复习一下梯度下降的几个方式了
    epoch_num   = 300
    loss        = 0.0 # 使用交叉熵
    ones        = np.ones(row_num)

    for i in range(epoch_num):
        #w_grad  = np.zeros(feature_num)
        #b_grad  = 0
        z       = np.dot(X_train, w) # [y1, y2, y3 ... ym] 1 - m: 1 - row_num

        sig     = sigmoid(z) # [z1, z2, .., zm]
        #print(sig)
        w_grad  = (-1) * np.dot(Y_train - sig, X_train)
        # 另一种思路
        #w_grad  = -1 * X_train * (Y_train - sig).reshape((row_num, 1))
        #w_grad = np.sum(w_grad, axis = 0)
        b_grad  = np.sum((-1) * (Y_train - sig), axis = 0)

        loss    += (-1) * (np.dot(Y_train, np.log(z)) + np.dot(1 - Y_train, np.log(1 - z)))
        w = w - l_rate * w_grad
        b = b - l_rate * b_grad
#         print(w, b)
        
    #print(sig)
    
    return w, b

w, b = train(X_train_set, Y_train_set)
print(w.shape, b)

/opt/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/opt/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in log


(106,) -19.50739019048858


In [24]:
# 使用批次，这个需要回看之前的作业和内容

# 其实就是将训练集分好几个部分(step) 然后开始
def train_batch(X_train, Y_train):
    sample_num  = X_train.shape[0]
    feature_num = X_train.shape[1]
    
    batch_num   = 32 # 一个批次的样本数量
    step_num    = int(sample_num / batch_num)
    epoch_num   = 300 # 训练轮数
    # 所有样本都被弄了一遍，就算一轮训练
    # step_num * batch_num = sample_num - > 1 epoch
    
    loss        = 0.0
    w           = np.zeros(feature_num)
    b           = 0.0
    l_rate      = 0.001
    for i in range(epoch_num):
        #w_grad = np.zeros(feature_num)
        #b_grad = 0.0
        loss = 0.0
        for step in range(step_num):
            X = X_train[ step * batch_num : (step + 1) * batch_num ]
            Y = Y_train[ step * batch_num : (step + 1) * batch_num ]
            #print(1, X)
            
            
            sig = sigmoid(np.dot(X, w) + b) # [z1, z2, .. z32].T
#             print(2, sig)
            cross_entropy = -1 * (np.dot(Y, np.log(sig)) + np.dot(1 - Y, np.log(1 - sig))) #交叉熵
            loss          += cross_entropy # 贯穿一次训练，loss 需要计算
            w_grad        = -1 * np.dot(Y - sig, X) 
            b_grad        = -1 * np.sum(Y - sig, axis = 0)
#             print(w_grad)
#             print(b_grad)
            
            # 一个批次中就要更新
            w   -=  l_rate * w_grad
            b   -=  l_rate * b_grad
            
        print(i, loss)
        # 放这里，就和 train 没差别了
#         w   -=  l_rate * w_grad
#         b   -=  l_rate * b_grad
        #print(w, b)
    
    return w, b
    
    
    
    
    

In [25]:
w_batch, b_batch = train_batch(X_train_set, Y_train_set)
print(w_batch, b_batch)

0 10907.366389520901
1 9665.948389430407
2 9538.865978287055
3 9484.73307676496
4 9455.569682167556
5 9438.166651542097
6 9427.106783812156
7 9419.746868740081
8 9414.66648630294
9 9411.051948771617
10 9408.41459394537
11 9406.44939800234
12 9404.95912804072
13 9403.811835704888
14 9402.916447692702
15 9402.208486466981
16 9401.641463638562
17 9401.181447703488
18 9400.803459028948
19 9400.488981031383
20 9400.224199343806
21 9399.998739380699
22 9399.804754646359
23 9399.636265236619
24 9399.488676015662
25 9399.358424359587
26 9399.242721654256
27 9399.139362862119
28 9399.046585677508
29 9398.96296593867
30 9398.887339637124
31 9398.818744506836
32 9398.756376073803
33 9398.699554427834
34 9398.647698964634
35 9398.600309072828
36 9398.556949274594
37 9398.517237709184
38 9398.480837143024
39 9398.447447888699
40 9398.416802178526
41 9398.388659650314
42 9398.362803687502
43 9398.339038421967
44 9398.317186251279
45 9398.29708576112
46 9398.278589968053
47 9398.261564817572
48 9398.

In [34]:
# 验证一下
#X_train_valid_set Y_train_valid_set
sig = sigmoid(np.dot(X_train_valid_set, w) + b)
x_sig = np.around(sig)
valid = (Y_train_valid_set == x_sig).astype(np.int)
valid.sum() / valid.shape[0]


0.7675061425061425

In [35]:
sig = sigmoid(np.dot(X_train_valid_set, w_batch) +  b_batch)
x_sig = np.around(sig)
valid = (Y_train_valid_set == x_sig).astype(np.int)
valid.sum() / valid.shape[0]

0.8525798525798526

In [40]:

test_data = pd.read_csv('data/test.csv')
ans       = pd.read_csv('data/correct_answer.csv')

x_test = x_data_treatment(test_data, False)
x_test
test_sig = sigmoid(np.dot(x_test, w_batch) +  b_batch)
test_sig = np.around(test_sig)
r = (test_sig == ans.label.values).astype("int64")
r.sum() / r.shape[0]

0.85148332412014

## 注意一下这个属性，每一行都对应乘上imi

In [11]:
a = np.array([[1,2,3,4],
              [1,2,3,4],
             [1,2,3,4]])
b = np.array([2,2,2,2])

a * b



array([[2, 4, 6, 8],
       [2, 4, 6, 8],
       [2, 4, 6, 8]])

In [12]:
np.array(1 + b)
(-1 * b)

array([-2, -2, -2, -2])

## 分割测试集
获得测试集之后，可以在其基础上分割**有效集与测试集**

In [13]:
#def split_train_valid(train_data, valid_percent):
    